#### Initialize ####

In [1]:
function tf {aws-vault exec terraform_user -- terraform $args}

function prompt_to_confirm { 
    $user_response = Read-host "Continue? y/n or 1/0"
    if ($user_response -ne "y" -and $user_response -ne 1) {break}
}

In [21]:
$aws_region = "us-east-2"
$env:AWS_REGION=$aws_region #for aws_vault
$env:TF_VAR_aws_region=$aws_region #for terraform

In [ ]:
terraform init #-upgrade

#### Workflow ####

In [ ]:
terraform fmt

In [ ]:
terraform validate

In [33]:
#Enable egress temporarily to install software after deployment
$env:TF_VAR_enable_egress=1

In [ ]:
tf plan -out=tfplan
terraform show -json tfplan > plan.json

In [ ]:
#Output planned resource addresses and their names in the cloud

$plan = Get-Content -Raw -Path "plan.json" | ConvertFrom-Json

$resourcesToDeploy = $plan.resource_changes | Where-Object { 
    $_.change.actions -contains "create" -or $_.change.actions -contains "update" 
} | ForEach-Object { 
    "Type: " + $_.type
    "TF name: " + $_.name
    "Cloud name: " + $_.change.after.name
    "Name tag: " + $_.change.after.tags_all.name
    ""
}

$resourcesToDeploy
Get-Date

In [35]:
###      ###
### PLAN ###
###      ###

tf plan
Get-Date

data.aws_ami.ami: Reading...
module.hello_world_app.data.aws_vpc.existing_vpc: Reading...
module.hello_world_app.module.asg.data.aws_ec2_instance_type.instance: Reading...
module.hello_world_app.module.asg.data.aws_ec2_instance_type.instance: Read complete after 1s [id=t2.micro]
data.aws_ami.ami: Read complete after 1s [id=ami-09da212cf18033880]
module.hello_world_app.data.aws_vpc.existing_vpc: Read complete after 2s [id=vpc-0deaf554d8149cc66]
module.hello_world_app.data.aws_subnets.vpc_subnets: Reading...
module.hello_world_app.data.aws_subnets.vpc_subnets: Read complete after 0s [id=us-east-2]
module.hello_world_app.data.aws_subnet.subnets["subnet-089fd6c9ad7ecdd2e"]: Reading...
module.hello_world_app.data.aws_subnet.subnets["subnet-053639f36fbe824cb"]: Reading...
module.hello_world_app.data.aws_subnet.subnets["subnet-0885bccad6bef5df1"]: Reading...
module.hello_world_app.data.aws_subnet.subnets["subnet-050a7a6df48d5a127"]: Reading...
module.hello_world_app.data.aws_subnet.subnets["s

In [36]:
###       ###
### APPLY ###
###       ###

#try {prompt_to_confirm} catch {break}
tf apply -auto-approve
Get-Date

module.hello_world_app.module.asg.data.aws_ec2_instance_type.instance: Reading...
data.aws_ami.ami: Reading...
module.hello_world_app.data.aws_vpc.existing_vpc: Reading...
module.hello_world_app.module.asg.data.aws_ec2_instance_type.instance: Read complete after 0s [id=t2.micro]
data.aws_ami.ami: Read complete after 1s [id=ami-09da212cf18033880]
module.hello_world_app.data.aws_vpc.existing_vpc: Read complete after 1s [id=vpc-0deaf554d8149cc66]
module.hello_world_app.data.aws_subnets.vpc_subnets: Reading...
module.hello_world_app.data.aws_subnets.vpc_subnets: Read complete after 0s [id=us-east-2]
module.hello_world_app.data.aws_subnet.subnets["subnet-0885bccad6bef5df1"]: Reading...
module.hello_world_app.data.aws_subnet.subnets["subnet-050a7a6df48d5a127"]: Reading...
module.hello_world_app.data.aws_subnet.subnets["subnet-089fd6c9ad7ecdd2e"]: Reading...
module.hello_world_app.data.aws_subnet.subnets["subnet-0e7ced3a3063a9760"]: Reading...
module.hello_world_app.data.aws_subnet.subnets["s

In [ ]:
#Disable egress after deployment
$env:TF_VAR_enable_egress=0

In [ ]:
#Deploy again to remove egress security rule

try {prompt_to_confirm} catch {break}
tf apply -auto-approve
Get-Date

In [38]:
$public_ip = terraform output -raw public_ip
$http_port = terraform output -raw LB_http_port

In [41]:
(Invoke-WebRequest -Uri ("http://" + $public_ip + ":$LB_http_port")).Content

Invoke-WebRequest: No such host is known.


Error: Command failed: SubmitCode: (Invoke-WebRequest -Uri ("http://" + $public_ip +  ...

In [40]:
###         ###
### DESTROY ###
###         ###

#try {prompt_to_confirm} catch {break}
tf destroy -auto-approve
Get-Date

data.aws_ami.ami: Reading...
module.hello_world_app.data.aws_vpc.existing_vpc: Reading...
module.hello_world_app.module.asg.data.aws_ec2_instance_type.instance: Reading...
module.hello_world_app.module.asg.data.aws_ec2_instance_type.instance: Read complete after 0s [id=t2.micro]
data.aws_ami.ami: Read complete after 0s [id=ami-09da212cf18033880]
module.hello_world_app.data.aws_vpc.existing_vpc: Read complete after 2s [id=vpc-0deaf554d8149cc66]
module.hello_world_app.data.aws_subnets.vpc_subnets: Reading...
module.hello_world_app.module.alb.aws_security_group.alb: Refreshing state... [id=sg-0ed088692ead47bc8]
module.hello_world_app.module.asg.aws_security_group.instance: Refreshing state... [id=sg-0b45d6de71edb0dc8]
module.hello_world_app.aws_lb_target_group.asg: Refreshing state... [id=arn:aws:elasticloadbalancing:us-east-2:588738573172:targetgroup/hello-world-prod/a6e6f012f096f04e]
module.hello_world_app.module.alb.aws_security_group_rule.allow_all_outbound: Refreshing state... [id=sg

#### Info ####

In [ ]:
terraform version

In [ ]:
#Root module configuration
cat .\main.tf

In [ ]:
($resources = terraform state list) | Out-Host

In [ ]:
foreach ($resource in $resources) { terraform state show $resource }

In [ ]:
terraform output

#### Troubleshooting ####

In [ ]:
#$env:TF_LOG = "TRACE"
#$env:TF_LOG_PATH = "log.txt"
#$env:TF_LOG = ""

In [ ]:
tf apply -refresh-only -auto-approve